<a href="https://colab.research.google.com/github/kimdonggyu2008/deep_daiv_-/blob/main/Hybrid_Demucs_Music_Source_Separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Demucs from Colab

This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/)
This is only for separation with pre-trained models, not training!

You can either upload files manually (slow) or link your Google Drive account.

In [ ]:
#!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [2]:
!pip install demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.w

In [1]:
import demucs.separate
import numpy as np
import librosa, soundfile
import pandas as pd
import os

In [2]:
# Please BE VERY CAREFUL, this will link your entire drive.
# So don't edit code, except the one that says 'Customize the following options',
# or you might mess up your files.
# IF YOU DO NO WANT TO LINK DRIVE, please see below for an alternative!
#from google.colab import drive
#drive.mount('/gdrive')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/drive/MyDrive/제목없는 폴더/'
out_path = '/content/drive/MyDrive/제목없는 폴더'

In [5]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def silence_below_threshold(audio_data, threshold):
    # 절대값이 threshold보다 작은 값은 0으로 변환
    audio_data[np.abs(audio_data) < threshold] = 0
    return audio_data

# def separate(inp=None, outp=None):
#     inp = inp or in_path
#     outp = outp or out_path
#     cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
#     if mp3:
#         cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
#     if float32:
#         cmd += ["--float32"]
#     if int24:
#         cmd += ["--int24"]
#     if two_stems is not None:
#         cmd += [f"--two-stems={two_stems}"]
#     files = [str(f) for f in find_files(inp)]
#     if not files:
#         print(f"No valid audio files in {in_path}")
#         return
#     print("Going to separate the files:")
#     print('\n'.join(files))
#     print("With command: ", " ".join(cmd))
#     p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
#     copy_process_streams(p)
#     p.wait()
#     if p.returncode != 0:
#         print("Command failed, something went wrong.")


# def from_upload():
#     out_path = Path('separated')
#     in_path = Path('tmp_in')

#     if in_path.exists():
#         rmtree(in_path)
#     in_path.mkdir()

#     if out_path.exists():
#         rmtree(out_path)
#     out_path.mkdir()

#     uploaded = files.upload()
#     for name, content in uploaded.items():
#         (in_path / name).write_bytes(content)
#     separate(in_path, out_path)

# 보컬과 배경음악만 분리하도록 two_stems 옵션을 추가하여 수정
def separate_vocals_and_instrumental(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    model = 'htdemucs'  # 사용할 모델 (예: htdemucs)
    two_stems = 'vocals'  # 두 개의 stem으로 나누기: 보컬과 그 외
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]

    # 두 스템 옵션을 추가 (보컬과 인스트루멘탈)
    cmd += [f"--two-stems={two_stems}"]

    # 입력 오디오 파일을 찾기
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return

    # 실행할 명령어 출력
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))

    # 명령어 실행
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()

    if p.returncode != 0:
        print("Command failed, something went wrong.")

# 파일 업로드 후 보컬과 배경음악 분리 실행
def from_upload_vocals_and_instrumental():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)

    # 보컬과 인스트루멘탈만 분리 실행
    separate_vocals_and_instrumental(in_path, out_path)


In [6]:
import numpy as np
import wave
import struct
from scipy.signal import butter, lfilter
from pathlib import Path

# WAV 파일을 읽어들이는 함수
def read_wav(filename):
    with wave.open(filename, 'rb') as wav_file:
        params = wav_file.getparams()  # 오디오 파일의 매개변수
        n_channels, sampwidth, framerate, n_frames = params[:4]

        # 오디오 데이터를 읽어서 numpy 배열로 변환
        frames = wav_file.readframes(n_frames * n_channels)
        out = struct.unpack_from("%dh" % n_frames * n_channels, frames)
        out = np.array(out)

        return out, params

# WAV 파일을 쓰는 함수 (파일 이름에 'vocals' 또는 'instrumental' 추가)
def write_wav(output_folder, original_filename, audio_data, params, stem_type):
    n_channels, sampwidth, framerate, n_frames = params[:4]

    # 파일 이름에 보컬인지 인스트루멘탈인지 추가
    output_filename = f"{original_filename.stem}_{stem_type}.wav"
    output_filepath = Path(output_folder) / output_filename

    with wave.open(output_filepath, 'wb') as wav_file:
        wav_file.setparams(params)

        # numpy 배열을 bytes로 변환하여 파일에 씀
        frames = struct.pack("%dh" % len(audio_data), *audio_data)
        wav_file.writeframes(frames)

# 고역 차단 필터를 적용하는 함수
def high_pass_filter(audio_data, cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_audio = lfilter(b, a, audio_data)
    return filtered_audio

# 분리된 보컬에만 20Hz 이하 소리 제거를 적용하는 함수
def apply_high_pass_filter_to_vocals(output_folder, original_filename, cutoff=20):
    # 보컬 파일에 대해 필터 적용
    vocals_file = Path(output_folder) / f"{original_filename.stem}_vocals.wav"
    if vocals_file.exists():
        print(f"Processing vocals file: {vocals_file}")
        audio_data, params = read_wav(vocals_file)
        framerate = params[2]

        # 고역 차단 필터 적용 (cutoff 주파수는 20Hz)
        filtered_audio = high_pass_filter(audio_data, cutoff, framerate)

        # 보컬로 저장 (원본 파일 이름에 '_vocals' 추가)
        write_wav(output_folder, original_filename, filtered_audio.astype(np.int16), params, "vocals")

# 분리된 배경음악 파일에도 적용
def rename_instrumental_files(output_folder, original_filename):
    instrumental_file = Path(output_folder) / f"{original_filename.stem}_no_vocals.wav"
    if instrumental_file.exists():
        # 배경음악 파일 이름 변경 (원본 파일 이름에 '_instrumental' 추가)
        new_filename = f"{original_filename.stem}_instrumental.wav"
        new_filepath = instrumental_file.with_name(new_filename)
        instrumental_file.rename(new_filepath)
        print(f"Renamed instrumental file to: {new_filepath}")

# 보컬과 배경음악만 분리하고 보컬에만 20Hz 이하 소리를 제거하는 기능 추가
def separate_vocals_and_instrumental(inp=None, outp=None, cutoff=20):
    inp = inp or in_path
    outp = outp or out_path
    model = 'htdemucs'  # 사용할 모델 (예: htdemucs)
    two_stems = 'vocals'  # 두 개의 stem으로 나누기: 보컬과 그 외
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]

    # 두 스템 옵션을 추가 (보컬과 인스트루멘탈)
    cmd += [f"--two-stems={two_stems}"]

    # 입력 오디오 파일을 찾기
    files = [Path(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return

    # 실행할 명령어 출력
    print("Going to separate the files:")
    print('\n'.join([str(f) for f in files]))
    print("With command: ", " ".join(cmd))

    # 명령어 실행
    p = sp.Popen(cmd + [str(f) for f in files], stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()

    if p.returncode != 0:
        print("Command failed, something went wrong.")
        return

    # 보컬 및 배경음악 파일을 처리
    for file in files:
        # 보컬 파일에만 고역 차단 필터 적용 (20Hz 이하 제거)
        apply_high_pass_filter_to_vocals(outp, file)

        # 배경음악 파일 이름 변경
        rename_instrumental_files(outp, file)

    print(f"Applied high-pass filter: cutoff={cutoff} Hz to vocals and renamed instrumental files.")

# 파일 업로드 후 보컬과 배경음악 분리 및 보컬에만 20Hz 이하 소리 제거 실행
def from_upload_vocals_and_instrumental(cutoff=20):
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        # 파일을 바이트로 작성하려면 open() 메서드 사용
        with open(in_path / name, 'wb') as f:
            f.write(content)

    # 보컬과 인스트루멘탈 분리 및 보컬에만 20Hz 이하 소리 제거 실행
    separate_vocals_and_instrumental(in_path, out_path, cutoff)


In [7]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
# separate_vocals_and_instrumental()

In [8]:
separate_vocals_and_instrumental(cutoff=20)

Going to separate the files:
/content/drive/MyDrive/제목없는 폴더/기아_응원가_솔로1.wav
/content/drive/MyDrive/제목없는 폴더/기아_응원가_솔로2.wav
/content/drive/MyDrive/제목없는 폴더/기아_응원가_솔로3.wav
/content/drive/MyDrive/제목없는 폴더/기아_응원가_솔로4.wav
/content/drive/MyDrive/제목없는 폴더/NC_응원가_솔로1.wav
/content/drive/MyDrive/제목없는 폴더/NC_응원가_솔로2.wav
/content/drive/MyDrive/제목없는 폴더/NC_응원가_솔로3.wav
/content/drive/MyDrive/제목없는 폴더/NC_응원가_솔로4.wav
/content/drive/MyDrive/제목없는 폴더/NC_응원가_솔로5.wav
/content/drive/MyDrive/제목없는 폴더/NC_응원가_솔로6.wav
/content/drive/MyDrive/제목없는 폴더/삼성_응원가_솔로1.wav
/content/drive/MyDrive/제목없는 폴더/삼성_응원가_솔로2.wav
/content/drive/MyDrive/제목없는 폴더/삼성_응원가_솔로3.wav
/content/drive/MyDrive/제목없는 폴더/두산_응원가_솔로1.wav
/content/drive/MyDrive/제목없는 폴더/두산_응워

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 127MB/s]


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/drive/MyDrive/제목없는 폴더/htdemucs
Separating track /content/drive/MyDrive/제목없는 폴더/기아_응원가_솔로1.wav


100%|██████████████████████████████████████████████| 35.099999999999994/35.099999999999994 [00:04<00:00,  7.18seconds/s]


Separating track /content/drive/MyDrive/제목없는 폴더/기아_응원가_솔로2.wav


...

Separating track /content/drive/MyDrive/제목없는 폴더/두산_응원가_솔로2.wav


100%|██████████████████████████████████████████████| 140.39999999999998/140.39999999999998 [00:07<00:00, 19.00seconds/s]


Separating track /content/drive/MyDrive/제목없는 폴더/LG트윈스_응원가_솔로.wav


100%|██████████████████████████████████████████████████████████████████████████| 46.8/46.8 [00:02<00:00, 18.74seconds/s]


Applied high-pass filter: cutoff=20 Hz to vocals and renamed instrumental files.


In [9]:
import os
from pathlib import Path

def rename_wav_files_with_parent_folder(directory):
    """
    주어진 디렉터리 내의 모든 하위 폴더의 wav 파일에 대해 상위 폴더 이름을 추가하여 파일 이름을 변경합니다.

    Parameters:
    - directory: 작업할 최상위 디렉터리 경로
    """
    directory_path = Path(directory)

    # 디렉터리 내 모든 하위 wav 파일을 검색
    for wav_file in directory_path.glob('**/*.wav'):
        parent_folder = wav_file.parent.name  # 상위 폴더 이름
        new_filename = f"{parent_folder}_{wav_file.name}"  # 상위 폴더 이름 + 기존 파일 이름
        new_filepath = wav_file.with_name(new_filename)  # 새로운 파일 경로

        # 파일 이름 변경
        wav_file.rename(new_filepath)
        print(f"Renamed: {wav_file} -> {new_filepath}")

# 사용 예시: 현재 디렉토리의 하위 폴더들에 대해 이름 변경
directory = '/content/drive/MyDrive/제목없는 폴더/htdemucs'  # 여기서 'your_directory_path'는 대상 디렉토리 경로
rename_wav_files_with_parent_folder(directory)


Renamed: /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/키움_응원가_솔로1_vocals.wav -> /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_키움_응원가_솔로1_vocals.wav
Renamed: /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/키움_응원가_솔로2_vocals.wav -> /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_키움_응원가_솔로2_vocals.wav
Renamed: /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/한화_응원가_솔로1_vocals.wav -> /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_한화_응원가_솔로1_vocals.wav
Renamed: /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/SSG_응원가_솔로_vocals.wav -> /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_SSG_응원가_솔로_vocals.wav
Renamed: /content/drive/MyDrive/제목없는 폴더/htdemucs/instrumentals/키움_응원가_솔로1_no_vocals.wav -> /content/drive/MyDrive/제목없는 폴더/htdemucs/instrumentals/instrumentals_키움_응원가_솔로

In [10]:
import shutil
from pathlib import Path

def move_audio_files(directory):
    """
    보컬 파일과 배경음악 파일을 각각 'vocals'와 'instrumentals' 폴더로 이동시키는 함수.

    Parameters:
    - directory: 작업할 최상위 디렉토리 경로
    """
    directory_path = Path(directory)
    vocals_path = directory_path / "vocals"
    instrumentals_path = directory_path / "instrumentals"

    # vocals 및 instrumentals 폴더 생성
    vocals_path.mkdir(exist_ok=True)
    instrumentals_path.mkdir(exist_ok=True)

    # 디렉터리 내 모든 하위 wav 파일을 검색 및 분류
    for wav_file in directory_path.glob('**/*.wav'):
        if "_vocals.wav" in wav_file.name:
            # 보컬 파일을 vocals 폴더로 이동
            destination = vocals_path / wav_file.name
            shutil.move(str(wav_file), str(destination))
            print(f"Moved {wav_file} to {destination}")
        elif "_no_vocals.wav" in wav_file.name:
            # 배경음악 파일을 instrumentals 폴더로 이동
            destination = instrumentals_path / wav_file.name
            shutil.move(str(wav_file), str(destination))
            print(f"Moved {wav_file} to {destination}")

# 사용 예시: 특정 디렉토리 내의 보컬 파일과 배경음악 파일을 각각 이동
directory = '/content/drive/MyDrive/제목없는 폴더/htdemucs'
move_audio_files(directory)


Moved /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_키움_응원가_솔로1_vocals.wav to /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_키움_응원가_솔로1_vocals.wav
Moved /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_키움_응원가_솔로2_vocals.wav to /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_키움_응원가_솔로2_vocals.wav
Moved /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_한화_응원가_솔로1_vocals.wav to /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_한화_응원가_솔로1_vocals.wav
Moved /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_SSG_응원가_솔로_vocals.wav to /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/vocals_SSG_응원가_솔로_vocals.wav
Moved /content/drive/MyDrive/제목없는 폴더/htdemucs/instrumentals/instrumentals_키움_응원가_솔로1_no_vocals.wav to /content/drive/MyDrive/제목없는 폴더/htdemucs/vocals/instrumentals_